In [1]:
import os
from seabird.cnv import fCNV
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [14]:

profile = fCNV('CNV/FIS.cnv')

profile.attrs # It will return the header, as a dictionary.

profile.keys() # It will list the available variables.

## name 8 = wetStar: Fluorescence, WET Labs WETstar [mg/m^3]

#profile['wetStar'] # If TEMP2 was on the .keys(), this is how you get the data. It will be a masked array.

['DEPTH',
 'PRES',
 'CNDC',
 'PSAL',
 'TEMP',
 'density',
 'sbox0Mm/Kg',
 'ph',
 'wetStar',
 'flag']

In [11]:
# directory where .cnv files are saved
datadir = 'CNV'

stnCounter=0
stnnames=[]
# loop through all cnv files
for filename in os.listdir(datadir):
    if filename.endswith(".cnv"):
        # keep count of number of files
        stnCounter += 1
        
        # combine path and file name in string
        pathfile = datadir+'/'+filename
        #print(pathfile)
        
        # create a list of station names
        #m = filename.index('_')
        #stnnames.append(filename[:m])
        #print(stnnames)
        
        # use seabird software to read csv file
        cast = fCNV(pathfile)
    
        latitude, longitude    = cast.attributes['LATITUDE'], cast.attributes['LONGITUDE']
    
        pressure    = cast['PRES']
        temperature = cast['TEMP'] 
        salinity    = cast['PSAL']
        #add on by ewaldo
        fluorescence = cast['wetStar']
        
        if stnCounter == 1:
            Ts = pd.Series(data=cast['TEMP'],index=cast['PRES'],name=cast)
            Ss = pd.Series(data=cast['PSAL'],index=cast['PRES'],name=cast)
            pHs = pd.Series(data=cast['ph'],index=cast['PRES'],name=cast)
        else:
            Ts = pd.concat([Ts,pd.Series(data=cast['TEMP'],index=cast['PRES'],name=cast)],axis=1)
            Ss = pd.concat([Ss,pd.Series(data=cast['PSAL'],index=cast['PRES'],name=cast)],axis=1)   
            pHs = pd.concat([pHs,pd.Series(data=cast['ph'],index=cast['PRES'],name=cast)],axis=1)

        lons, lats = [],[]
        
        # Attributes to feed data array
        lons.append(latitude)
        lats.append(longitude)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
# Dataframe to DataArray
Txr = Ts.stack().to_xarray().rename({'level_0' : 'pressure', 'level_1' : 'cast'})
Txr.name = 'temperature'
Sxr = Ss.stack().to_xarray().rename({'level_0' : 'pressure', 'level_1' : 'cast'})
Sxr.name = 'salinity'
pHxr = pHs.stack().to_xarray().rename({'level_0' : 'pressure', 'level_1' : 'cast'})
pHxr.name = 'pH'

In [ ]:
ctdcombined = xr.merge([Txr,Sxr,pHxr])
#ctdcombined

In [ ]:
ctdcombined.pressure.attrs['units'] = 'dbar'
ctdcombined.temperature.attrs['units'] = 'degree C'
ctdcombined.salinity.attrs['units'] = 'psu'

In [ ]:

ylim = (ctdcombined.pressure.values.max(),0)
plt.rcParams.update({'font.size': 16})

plt.figure(figsize=(6,10))
for i in range(stnCounter):
    ctdcombined.isel(cast=i).plot.scatter('salinity','pressure',label=stnnames[i])
plt.legend(loc=4)
plt.xlim([29, 34.5])
plt.ylim(*ylim)
plt.title("Cruise 1 salinity profiles")

In [ ]:
plt.figure(figsize=(6,10))
for i in range(stnCounter):
    ctdcombined.isel(cast=i).plot.scatter('temperature','pressure',label=stnnames[i])
plt.legend(loc=3)
plt.xlim([16, 20.5])
plt.ylim(*ylim)
plt.title("Cruise 1 temperature profiles")